In [2]:
import pandas as pd
import urllib, json
import re
import numpy as np
from difflib import SequenceMatcher
import time

In [3]:
count=0
def RT_ME_rating(id):
    global count
    url="http://www.omdbapi.com/?i="+str(id)
    count+=1
    if count%200==0:
        print count
    try:
        response = urllib.urlopen(url)
        data = json.loads(response.read())
        me=""
        rt=""
        for i in range(len(data['Ratings'])):
            if data["Ratings"][i]['Source']=="Metacritic":
                me = data['Ratings'][i]['Value'][:-4]
            if data["Ratings"][i]['Source']=="Rotten Tomatoes":
                rt = data["Ratings"][i]['Value'][:-1]
        return rt+"_"+me
    except ValueError:
        print "the ID is", id
        return "_"

In [4]:
count=0
def douban_rating(title):
    try:
        url = 'http://api.douban.com/v2/movie/search?q='+title.replace(" ", "+")
        response = urllib.urlopen(url)
        data = json.loads(response.read())
        #print data
        df = pd.DataFrame()
        length = len(data['subjects'])
        for i in range(length):
            new_data=data['subjects'][i]
            new_title = new_data['original_title']
            new_score = new_data['rating']['average']
            df = df.append({'score': new_score, 'title': new_title}, ignore_index=True)
        df = df.sort(columns='score', ascending=False).drop_duplicates('title')
        df['likelihood']=df['title'].apply(lambda x: SequenceMatcher(None, x, title).ratio())
        return float(df[df['likelihood']==df['likelihood'].max()]['score'])
    except ValueError as e:
        print "Value Error:", title, e.message
        return 0.0
    except KeyError as e:
        print "KeyError:", title, e.message
        return 0.0
    except TypeError as e:
        print "TypeError:", title, e.message
        return 0.0

In [43]:
films=pd.read_csv("./movie_final.csv", index_col=0)

In [36]:
films.loc[films['rt_score']==0, 'rt_score']=np.nan

In [37]:
len(films[films['rt_score']==0])

0

In [42]:
films.to_csv("./movie_final.csv")